In [1]:
import pandas as pd


In [7]:
df = pd.read_csv("../output_csv/combine/solenopsis_invicta.csv")

In [32]:
time = sorted(list(set(df.Time)))

In [42]:
data = []
for t in time:
    d = df[df.Time==t][['lat','long','EI']].values.tolist()
    data.append(d)

In [43]:

import folium
import folium.plugins as plugins
import numpy as np



In [44]:
m = folium.Map([20., 121.], tiles='stamentoner', zoom_start=6)

hm = plugins.HeatMapWithTime(
    data,
    index=time,
    auto_play=True,
    max_opacity=0.3
)

hm.add_to(m)

m